In [2]:
import math
import numpy as np
import pandas as pd
import datetime as dt

In [26]:
S0 = 100
K = 98
T = [0,0.25,0.5,0.75,1]
dt = 0.25
r = 0
delta = 0.02
#q1 sigma cst = 23%
sigma = 0.23
mcpath = 1000
n = len(T)
#matrix N*5


In [82]:
vol = 0.23
var = vol ** 2
dailyVol = np.sqrt( var / dt )
time=1
dt=0.25
inter = time * dt

In [138]:
S = np.zeros((mcpath,n))

In [142]:
for i in range(0,mcpath):
    t = 0 
    W = 0 
    C=np.ones(5)
    for j in range(0,n):
        if j==0:
            C[j]=100
        else:
            W = np.random.normal(0,np.sqrt(dt))
            C[j]=np.exp((r-0.5*sigma**2)*dt+sigma*W)
        S[i,j] = np.prod(C)